In [ ]:
import datetime
import presto

from datetime import timedelta
from jinja2 import Template
from presto import transaction

In [ ]:
# Defaults to fall back to
date = datetime.date.today() - timedelta(days=365)

# Replace these so we can see the override
drop_query = '../../../resources/jasper/demo/drop.sql'
ctas_query = '../../../resources/jasper/demo/ctas.sql'

In [ ]:
conn = presto.dbapi.connect(
    host='presto-1.prod.dataf.eb',
    port=8080,
    user='jasper',
    catalog='hive',
    isolation_level=transaction.IsolationLevel.REPEATABLE_READ,
)
cur = conn.cursor()

In [ ]:
def template_query(file_path, context={}):
    lines = '\n'.join(open(file_path, 'r').readlines())
    template = Template(lines)
    return template.render(**context)

In [ ]:
parameters = {
    'date': date.strftime('%Y-%m-%d')
}
drop = template_query(drop_query)
ctas = template_query(ctas_query, parameters)

In [ ]:
cur.execute(drop)
if not cur.fetchall():
    raise Exception("Table did not drop succesfully")
conn.commit()

In [ ]:
cur.execute(ctas)
num_rows = cur.fetchall()
conn.commit()

In [ ]:
print(num_rows)